In [ ]:
# 基于循环神经网络实现mnist手写数字识别。
class RNN_Net(t.nn.Module):
    def __init__(self):
        super(RNN_Net, self).__init__()
        self.rnn_layers = t.nn.RNN(input_size=28, hidden_size=100, 
                                   num_layers=2, batch_first=True)
        self.out_layer = t.nn.Linear(100, 10)

    def forward(self, x):
        r_out, hn = self.rnn_layers(x, None)
        out = self.out_layer(r_out[:, -1, :])
        return out
    
    
class LSTM_Net(t.nn.Module):
    def __init__(self):
        super(LSTM_Net, self).__init__()
        self.rnn_layers = t.nn.LSTM(input_size=28, hidden_size=64, 
                                    num_layers=1, batch_first=True)
        self.out_layer = t.nn.Linear(64, 10)

    def forward(self, x):
        r_out, hn = self.rnn_layers(x, None)
        out = self.out_layer(r_out[:, -1, :])
        return out
    
rnn_model = LSTM_Net()
print(rnn_model)
optimizer = t.optim.Adam(rnn_model.parameters(), lr=0.001)
loss_fn = t.nn.CrossEntropyLoss()

for epoch in range(2):
    for i, (x_train, y_train) in enumerate(train_dl):
        x_train = x_train.view(-1, 28, 28)
        y_pred = rnn_model(x_train)
        train_loss = loss_fn(y_pred, y_train)
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()
        if (i + 1) % 50 == 0:
            print(i + 1, train_loss.item())

total = 0
correct_count = 0
for test_images, test_labels in test_dl:
    for i in range(len(test_labels)):
        image = test_images[i].view(1, 28, 28)
        with t.no_grad():
            pred_labels = rnn_model(image)
        probs = list(pred_labels.numpy()[0])
        pred_label = probs.index(max(probs))
        true_label = test_labels.numpy()[i]
        if pred_label == true_label:
            correct_count += 1
        total += 1
print("total acc : %.2f\n"%(correct_count / total))
t.save(rnn_model, './rnn_mnist_model.pt')